In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras import datasets, applications

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#use this string for the path to the folder containing all the images in kaggle

folder_path = '/content/drive/MyDrive/Semester2/Deep_learning/PROJECT_DEEP_LEARNING/'

class_indices_file = open('/content/drive/MyDrive/Semester2/Deep_learning/PROJECT_DEEP_LEARNING/ucfTrainTestlist_recogintion/classInd.txt', 'r')
class_index = {}
for line in class_indices_file:
    class_ = line.split(' ')
    class_index[class_[1].strip('\n')] = int(class_[0])

class_indices_file.close()

print(class_index)

{'ApplyEyeMakeup': 1, 'ApplyLipstick': 2, 'Archery': 3, 'BabyCrawling': 4, 'BalanceBeam': 5, 'BandMarching': 6, 'BaseballPitch': 7, 'Basketball': 8, 'BasketballDunk': 9, 'BenchPress': 10, 'Biking': 11, 'Billiards': 12, 'BlowDryHair': 13, 'BlowingCandles': 14, 'BodyWeightSquats': 15, 'Bowling': 16, 'BoxingPunchingBag': 17, 'BoxingSpeedBag': 18, 'BreastStroke': 19, 'BrushingTeeth': 20, 'CleanAndJerk': 21, 'CliffDiving': 22, 'CricketBowling': 23, 'CricketShot': 24, 'CuttingInKitchen': 25, 'Diving': 26, 'Drumming': 27, 'Fencing': 28, 'FieldHockeyPenalty': 29, 'FloorGymnastics': 30, 'FrisbeeCatch': 31, 'FrontCrawl': 32, 'GolfSwing': 33, 'Haircut': 34, 'Hammering': 35, 'HammerThrow': 36, 'HandstandPushups': 37, 'HandstandWalking': 38, 'HeadMassage': 39, 'HighJump': 40, 'HorseRace': 41, 'HorseRiding': 42, 'HulaHoop': 43, 'IceDancing': 44, 'JavelinThrow': 45, 'JugglingBalls': 46, 'JumpingJack': 47, 'JumpRope': 48, 'Kayaking': 49, 'Knitting': 50, 'LongJump': 51, 'Lunges': 52, 'MilitaryParade': 

In [ ]:
req_classes = [[1,2], [8,9], [23,24], [30], [87], [70], [72], [100], [59,60,61,62,63,64,65,66,67], 
               [83],[98], [29], [85], [79], [80], [84], [51], [45], [77], [73], [44]]

class_names = []
for _ in req_classes:
    classes = []
    for __ in _:
      classes.append([key for key in list(class_index.keys()) if class_index[key] == __][0])
    class_names.append(classes)
print(class_names)


[['ApplyEyeMakeup', 'ApplyLipstick'], ['Basketball', 'BasketballDunk'], ['CricketBowling', 'CricketShot'], ['FloorGymnastics'], ['SumoWrestling'], ['PullUps'], ['PushUps'], ['WritingOnBoard'], ['PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'PlayingViolin'], ['SkyDiving'], ['WalkingWithDog'], ['FieldHockeyPenalty'], ['SoccerPenalty'], ['Shotput'], ['SkateBoarding'], ['SoccerJuggling'], ['LongJump'], ['JavelinThrow'], ['SalsaSpin'], ['Rafting'], ['IceDancing']]


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
label_dict = {}

label = 1
for _ in req_classes:
    for id in _:
        label_dict[id] = label
    label += 1

print(label_dict)

{1: 1, 2: 1, 8: 2, 9: 2, 23: 3, 24: 3, 30: 4, 87: 5, 70: 6, 72: 7, 100: 8, 59: 9, 60: 9, 61: 9, 62: 9, 63: 9, 64: 9, 65: 9, 66: 9, 67: 9, 83: 10, 98: 11, 29: 12, 85: 13, 79: 14, 80: 15, 84: 16, 51: 17, 45: 18, 77: 19, 73: 20, 44: 21}


In [ ]:
#considering only one test train split 01
train_path_file = open("/content/drive/MyDrive/Semester2/Deep_learning/PROJECT_DEEP_LEARNING/ucfTrainTestlist_recogintion/trainlist01.txt", "r")
train_path_list = []
train_y = []
# print(next(train_path_file))
list_req_class = [i for l in req_classes for i in l]
print(list_req_class)
for _ in train_path_file:
    path, class_num = _.split(' ')
    if (int(class_num.strip('\n')) in list_req_class):
        train_path_list.append(path)
        train_y.append(label_dict[int(class_num.strip('\n'))])
        #print(int(label_dict[class_num.strip('\n')]))
    
train_path_file.close()


[1, 2, 8, 9, 23, 24, 30, 87, 70, 72, 100, 59, 60, 61, 62, 63, 64, 65, 66, 67, 83, 98, 29, 85, 79, 80, 84, 51, 45, 77, 73, 44]


In [ ]:
len(train_path_list)

3035

In [ ]:
test_path_file = open("/content/drive/MyDrive/Semester2/Deep_learning/PROJECT_DEEP_LEARNING/ucfTrainTestlist_recogintion/testlist01.txt", "r")
test_path_list = []
test_y = []
# print(next(test_path_file))
for _ in test_path_file:
    path = _.strip('\n')
    a,b = path.split('/')
    if (class_index[a] in list_req_class):
        test_path_list.append(path)
        test_y.append(label_dict[class_index[a]])


test_path_file.close()



In [ ]:
len(test_path_list)

1214

In [ ]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
train_y_hot = pd.get_dummies(train_y)
test_y_hot = pd.get_dummies(test_y)

In [ ]:
labels = np.unique(np.array(train_y))

sig_frame_dict = {}
for i in labels:
    with open(folder_path+ 'train_hist_frames/'+ 'class_'+ str(i) +'_sig_frames.txt', 'r') as readfile:
        contents = readfile.read()
        lines = contents.splitlines()
    for l in lines:
        #print(l)
        l_arr = l.split(" ", 1)
        sig_frame_dict[folder_path + 'UCF-101/'+ l_arr[0]] = eval(l_arr[1])

    with open(folder_path + 'test_hist_frames/'+ 'test_class_'+ str(i) +'_sig_frames.txt', 'r') as read_testfile:
        contents = read_testfile.read()
        lines = contents.splitlines()
    for l in lines:
        l_arr = l.split(" ", 1)
        sig_frame_dict[folder_path+ 'UCF-101/' + l_arr[0]] = eval(l_arr[1])
    




In [ ]:
sig_frame_dict[folder_path + 'UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi']

[0, 1, 4, 6, 7, 12, 16, 22, 28, 33, 37, 42, 44, 48, 53, 58, 64, 72, 76, 84]

In [ ]:
# video1 = cv2.VideoCapture(folder_path+ 'UCF101/UCF-101/' +'Basketball/v_Basketball_g16_c06.avi')
# total_frames = video1.get(cv2.CAP_PROP_FRAME_COUNT)
# #     frames = [
# print(total_frames)
# #     while video1.isOpened():
# #         ret, frame = video1.read()
# #         if not ret:
# #             break
# #         frames.append(frame)
# #         if frame.shape != (240, 320, 3):
# #             print("Invalid Image")
# #     print("frame Length",len(frames))

# interval = int(total_frames) // 20
# print(interval)
# frame_count = 0
# frames_arr = []
# while video1.isOpened():
#     ret, frame = video1.read()

#     if not ret:  #reached end of video
#         print("break",frame_count)
#         break

#     if frame_count % interval == 0:
#         resized_frame = cv2.resize(frame, (120, 160))
#         normalized_frame = resized_frame / 255
#         frames_arr.append(normalized_frame)
#         print(frame_count)

#     if frame_count >= 19 * interval:
#         break

#     frame_count += 1

# print("frames",len(frames_arr))
# video1.release()

In [ ]:
# for path in train_path_list[224:240]:
#     video1 = cv2.VideoCapture(folder_path+ 'UCF101/UCF-101/' +path)
#     total_frames = video1.get(cv2.CAP_PROP_FRAME_COUNT)
# #     frames = []
# #     print(total_frames)
# #     while video1.isOpened():
# #         ret, frame = video1.read()
# #         if not ret:
# #             break
# #         frames.append(frame)
# #         if frame.shape != (240, 320, 3):
# #             print("Invalid Image")
# #     print("frame Length",len(frames))
    
#     interval = total_frames // 20
#     print(interval)
#     frame_count = 0
#     frames_arr = []
#     frames_total = []
#     while video1.isOpened():
#         ret, frame = video1.read()

#         if not ret:  #reached end of video
#             break
#         frames_total.append(frame)

#     video1.release()
#     total_frames = len(frames_total)
#     print(total_frames)
#     interval = total_frames // 20

#     for _ in range (total_frames):
#         if _ % interval == 0:
#             resized_frame = cv2.resize(frames_total[_], (120, 160))
#             normalized_frame = resized_frame / 255
#             frames_arr.append(normalized_frame)

#         if _ >= 19 * interval:
#             break
#     print('length',  len(frames_arr))

# ## **data loader**

## for skip interval/random/adaptive frame selection

In [26]:

class VideoGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X=train_path_list, Y=train_y, batch_size=32, frame_selection = 'skip'):
        'Initialization'
        self.batch_size = batch_size
        self.X = X
        self.Y = Y
        self.frame_selection = frame_selection
       
        
    # X_paths, y_labels = generate_X_Y(self.train)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        
        
        if (index == len(self.X)//self.batch_size - 1 and (index+1)*self.batch_size > len(self.X)-1):
            indexes = np.arange(index*self.batch_size,len(self.X), 1)
        else:
            indexes = np.arange((index)*self.batch_size,(index+1)*self.batch_size,1)
        # Find list of paths
        list_paths = [self.X[k] for k in indexes]
        
        # Generate data
        X_batch = []
        y_batch =[]

      
        for ind in indexes:
            vid_path = self.X[ind]
            frame_arr = self.generate_frames_from_videos(folder_path+ 'UCF-101/' + vid_path)
            X_batch.append(frame_arr)
            y_batch.append(self.Y[ind])
        
#         print(index, np.array(X_batch).shape)
        return applications.xception.preprocess_input(np.array(X_batch)), np.array(y_batch)

    def generate_frames_from_videos(self, video_path):

        #CAPTURING FRAMES after EVERY 20 FRAMES OF VIDEOS
  
        video1 = cv2.VideoCapture(video_path)
        #total_frames = video1.get(cv2.CAP_PROP_FRAME_COUNT)
        
        if (self.frame_selection == 'skip'):
            frames_total = []
            frames_arr = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)

            video1.release()
            total_frames = len(frames_total)
            #print(total_frames)
            interval = total_frames // 20

            for _ in range (total_frames):
                if _ % interval == 0:
                    resized_frame = cv2.resize(frames_total[_], (224, 224))
                    normalized_frame = resized_frame
                    frames_arr.append(normalized_frame)

                if _ >= 19 * interval:
                    break

#             print(np.array(frames_arr).shape)
            return frames_arr
        
        
        elif (self.frame_selection == 'random'):
            
            frames_total = []
            frames_arr = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)

            video1.release()
            total_frames = len(frames_total)
            rand_list = np.random.randint(0,total_frames+1,20)

            for _ in rand_list:
                resized_frame = cv2.resize(frames_total[_], (224, 224))
                normalized_frame = resized_frame
                frames_arr.append(normalized_frame)

            video1.release()
            return frames_arr
        
        
        elif (self.frame_selection == 'hist_difference'):
            frames_num = sig_frame_dict[video_path]
            #print(len(frames_num))
            frame_count = 0
            frames_arr = []
            
            frames_total = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)
                
            total_frames = len(frames_total)
            
            actual = []
            
            for i in range(len(frames_num)):
                ind = list(frames_num)[i]
                if ind < total_frames:
                    actual.append(ind)
                else:
                    int_ind = np.random.randint(0,total_frames)
                    actual.append(int_ind)
            if (len(actual)< 20):
                actual = actual + list(np.random.randint(0,total_frames,20-len(actual)))
            for ind in actual:
                resized_frame = cv2.resize(frames_total[int(ind)], (224, 224))
                #normalized_frame = resized_frame / 255
                frames_arr.append(tf.keras.applications.resnet50.preprocess_input(resized_frame))

            video1.release()
            #print(np.array(frames_arr).shape)
            return frames_arr
            


    

## pretrained Xception 

In [ ]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    classes = len(req_classes)
)
Xcep_pt_model.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [ ]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,120,160,3)))
model_pret_x.add(TimeDistributed(Dropout(0.25)))

model_pret_x.add(TimeDistributed(Flatten()))

model_pret_x.add(LSTM(32))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))




In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)


In [ ]:
model_pret_x.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (None, 20, 4, 5, 2048)   20861480  
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 20, 4, 5, 2048)   0         
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 20, 40960)        0         
 tributed)                                                       
                                                                 
 lstm_1 (LSTM)               (None, 128)               21037568  
                                                                 
 dense_1 (Dense)             (None, 21)                2709      
                                                      

In [ ]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 20, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

Epoch 1/20
94/94 [==============================] - 306s 3s/step - loss: 3.2905 - accuracy: 0.1130 - val_loss: 2.6779 - val_accuracy: 0.3057
Epoch 2/20
94/94 [==============================] - 295s 3s/step - loss: 2.6081 - accuracy: 0.3135 - val_loss: 2.5052 - val_accuracy: 0.3083
Epoch 3/20
94/94 [==============================] - 293s 3s/step - loss: 2.5120 - accuracy: 0.3115 - val_loss: 2.4241 - val_accuracy: 0.3176
Epoch 4/20
94/94 [==============================] - 294s 3s/step - loss: 2.4001 - accuracy: 0.3348 - val_loss: 2.3536 - val_accuracy: 0.3784
Epoch 5/20
94/94 [==============================] - 295s 3s/step - loss: 2.3215 - accuracy: 0.3674 - val_loss: 2.2873 - val_accuracy: 0.3733
Epoch 6/20
94/94 [==============================] - 292s 3s/step - loss: 2.1953 - accuracy: 0.3933 - val_loss: 2.2229 - val_accuracy: 0.3936
Epoch 7/20
94/94 [==============================] - 293s 3s/step - loss: 2.0787 - accuracy: 0.4362 - val_loss: 2.1613 - val_accuracy: 0.4257
Epoch 8/20
94

In [ ]:
model_pret_x.save("xception.h5")

## inception v3

In [ ]:
incep_pret_model = applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    classes = len(req_classes)
)
incep_pret_model.trainable = False

In [ ]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(incep_pret_model, input_shape= (20,120,160,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))





In [ ]:
model_pret_x.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 20, 2, 3, 2048)   21802784  
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 20, 2048)         0         
 tributed)                                                       
                                                                 
 lstm_2 (LSTM)               (None, 128)               1114624   
                                                                 
 dense_2 (Dense)             (None, 21)                2709      
                                                                 
Total params: 22,920,117
Trainable params: 1,117,333
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [ ]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1
                                                                                           ), metrics = ["accuracy"])

### skip interval model

### lr = 0.1, momentum = 0

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

In [ ]:
model_pret_x.save('/kaggle/working/model_incep_skip.h5')

### lr = 0.01, momentum= 0.1

In [ ]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01,
                                momentum = 0.1), metrics = ["accuracy"])

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

## **image size 224x224**

In [ ]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    classes = len(req_classes)
)
Xcep_pt_model.trainable = False

83683744/83683744 [==============================] - 1s 0us/step


In [ ]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))





In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)


In [ ]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])


In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 20, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

Epoch 1/20
94/94 [==============================] - 1454s 16s/step - loss: 2.7381 - accuracy: 0.2916 - val_loss: 2.2222 - val_accuracy: 0.3564
Epoch 2/20
94/94 [==============================] - 330s 4s/step - loss: 2.2065 - accuracy: 0.3860 - val_loss: 2.0920 - val_accuracy: 0.4966
Epoch 3/20
94/94 [==============================] - 318s 3s/step - loss: 1.9295 - accuracy: 0.4721 - val_loss: 1.7794 - val_accuracy: 0.5312
Epoch 4/20
94/94 [==============================] - 324s 3s/step - loss: 1.7092 - accuracy: 0.5180 - val_loss: 1.6970 - val_accuracy: 0.5355
Epoch 5/20
94/94 [==============================] - 319s 3s/step - loss: 1.5735 - accuracy: 0.5549 - val_loss: 1.4624 - val_accuracy: 0.5769
Epoch 6/20
94/94 [==============================] - 319s 3s/step - loss: 1.4185 - accuracy: 0.5954 - val_loss: 1.4405 - val_accuracy: 0.6351
Epoch 7/20
94/94 [==============================] - 373s 4s/step - loss: 1.2697 - accuracy: 0.6672 - val_loss: 1.3770 - val_accuracy: 0.6216
Epoch 8/20


In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

Epoch 1/10
94/94 [==============================] - 317s 3s/step - loss: 0.2423 - accuracy: 0.9697 - val_loss: 0.8485 - val_accuracy: 0.7644
Epoch 2/10
94/94 [==============================] - 319s 3s/step - loss: 0.2246 - accuracy: 0.9727 - val_loss: 0.8109 - val_accuracy: 0.7669
Epoch 3/10
94/94 [==============================] - 318s 3s/step - loss: 0.1925 - accuracy: 0.9774 - val_loss: 0.8087 - val_accuracy: 0.7644
Epoch 4/10
94/94 [==============================] - 322s 3s/step - loss: 0.1722 - accuracy: 0.9860 - val_loss: 0.7951 - val_accuracy: 0.7449
Epoch 5/10
94/94 [==============================] - 368s 4s/step - loss: 0.1615 - accuracy: 0.9847 - val_loss: 0.8950 - val_accuracy: 0.7390
Epoch 6/10
94/94 [==============================] - 363s 4s/step - loss: 0.1481 - accuracy: 0.9854 - val_loss: 0.7857 - val_accuracy: 0.7762
Epoch 7/10
94/94 [==============================] - 302s 3s/step - loss: 0.1167 - accuracy: 0.9924 - val_loss: 0.7464 - val_accuracy: 0.7779
Epoch 8/10
94

In [ ]:
model_pret_x.save(folder_path + "xcep_224_skip_model.h5")

## pretrained vgg 224 224 random

In [ ]:
vgg_pret_model = applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    classes = len(req_classes)
)
vgg_pret_model.trainable = False

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(vgg_pret_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))






In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)



In [ ]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])



In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'random'),epochs = 2, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'random'),callbacks = [early_stopping_callback]) 

## Pretrained ResNet

In [ ]:
resnet_pt_model = tf.keras.applications.resnet.ResNet50(
    include_top=False,
    weights='imagenet'
)
resnet_pt_model.trainable = False


94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(resnet_pt_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

# model_pret_x.add(Dense(128, activation = 'sigmoid'))

# model_pret_x.add(Dense(64, activation = 'sigmoid'))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))




In [19]:
model_pret_x.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 7, 7, 2048)   23587712  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 2048)         0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 128)               1114624   
                                                                 
 dense (Dense)               (None, 21)                2709      
                                                                 
Total params: 24,705,045
Trainable params: 1,117,333
Non-trainable params: 23,587,712
_________________________________________________________________


In [20]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 20, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 